In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os
import glob
import PIL
from PIL import Image
from torch.utils import data as D
from torch.utils.data.sampler import SubsetRandomSampler
import random
import torchsummary
import time
import tensorflow as tf

############### Miok ############
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from datetime import datetime
from torch.autograd import Variable

CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
MILESTONES = [20, 40, 80] #learning rate 값을 60 epoch, 120 epoch, 160 epoch 별로 어떻게 하겠따.
CHECKPOINT_PATH = 'checkpoint'
TIME_NOW = datetime.now().isoformat()

####### onnx ################
#import io
#import torch.onnx
#import onnx
#import onnxruntime as ort


In [12]:
print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

1.5.0+cu101
cuda:0


In [0]:
batch_size = 64
validation_ratio = 0.1
random_seed = 10
initial_lr = 0.1
num_epoch = 1
############
num_workers = 4
EPOCH = 100
lr = 0.1
warm = 1
SAVE_EPOCH=10

In [0]:
 class conv_bn_relu(nn.Module):
    def __init__(self, nin, nout, kernel_size, stride, padding,groups=1, bias=False):
        super(conv_bn_relu, self).__init__()
        self.conv = nn.Conv2d(nin, nout, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups,bias=bias)
        self.batch_norm = nn.BatchNorm2d(nout)
        self.relu = nn.ReLU(True)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = self.relu(out)

        return out

In [0]:
class Transition_layer(nn.Sequential):
  def __init__(self, nin, theta=1):    
      super(Transition_layer, self).__init__()
      
      self.add_module('conv_1x1', conv_bn_relu(nin=nin, nout=int(nin*theta), kernel_size=1, stride=1, padding=0, bias=False))
      self.add_module('avg_pool_2x2', nn.AvgPool2d(kernel_size=2, stride=2, padding=0))

In [0]:
################## Miok
class ShuffleBlock(nn.Module):
    def __init__(self, groups):
        super(ShuffleBlock, self).__init__()
        self.groups = groups

    def forward(self, x):
        '''Channel shuffle: [N,C,H,W] -> [N,g,C/g,H,W] -> [N,C/g,g,H,w] -> [N,C,H,W]'''
        N,C,H,W = x.size()
        g = self.groups
        return x.view(N,g,C//g,H,W).permute(0,2,1,3,4).reshape(N,C,H,W)
############################

In [0]:
class StemBlock(nn.Module):
    def __init__(self):
        super(StemBlock, self).__init__()
        #################### Miok ####################
        self.conv_3x3_first = conv_bn_relu(nin=3, nout=32, kernel_size=3, stride=2, padding=1, bias=False)
        
        self.conv_1x1_left = conv_bn_relu(groups=4,nin=32, nout=16, kernel_size=1, stride=1, padding=0, bias=False)
        self.shuffle1 = ShuffleBlock(groups=4)
        self.conv_3x3_left = conv_bn_relu(nin=16, nout=32, kernel_size=3, stride=2, padding=1, bias=False)
        
        self.max_pool_right = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.conv_1x1_last = conv_bn_relu(groups=4,nin=64, nout=32, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        out_first = self.conv_3x3_first(x)
        
        out_left = self.conv_1x1_left(out_first)
        out_left = self.shuffle1(out_left)
        out_left = self.conv_3x3_left(out_left)
        
        out_right = self.max_pool_right(out_first)
        
        out_middle = torch.cat((out_left, out_right), 1)
        
        out_last = self.conv_1x1_last(out_middle)
                
        return out_last

In [0]:
class dense_layer(nn.Module):
  def __init__(self, nin, growth_rate, drop_rate=0.2):    
      super(dense_layer, self).__init__()
      
      self.dense_left_way = nn.Sequential()
      #################### Miok ####################
      self.dense_left_way.add_module('conv_1x1', conv_bn_relu(groups=4,nin=nin, nout=growth_rate*2, kernel_size=1, stride=1, padding=0, bias=False))
      self.shuffle2 = ShuffleBlock(groups=4)
      self.dense_left_way.add_module('conv_3x3', conv_bn_relu(nin=growth_rate*2, nout=growth_rate//2, kernel_size=3, stride=1, padding=1, bias=False))
            
      self.dense_right_way = nn.Sequential()
      
      self.dense_right_way.add_module('conv_1x1', conv_bn_relu(groups=4,nin=nin, nout=growth_rate*2, kernel_size=1, stride=1, padding=0, bias=False))
      self.shuffle3 = ShuffleBlock(groups=4)
      self.dense_right_way.add_module('conv_3x3_1', conv_bn_relu(nin=growth_rate*2, nout=growth_rate//2, kernel_size=3, stride=1, padding=1, bias=False))
      self.dense_right_way.add_module('conv_3x3 2', conv_bn_relu(nin=growth_rate//2, nout=growth_rate//2, kernel_size=3, stride=1, padding=1, bias=False))
      
      self.drop_rate = drop_rate
      
  def forward(self, x):
      left_output = self.dense_left_way(x)
      right_output = self.dense_right_way(x)

      if self.drop_rate > 0:
          left_output = F.dropout(left_output, p=self.drop_rate, training=self.training)
          right_output = F.dropout(right_output, p=self.drop_rate, training=self.training)
          
      dense_layer_output = torch.cat((x, left_output, right_output), 1)
            
      return dense_layer_output

In [0]:
class DenseBlock(nn.Sequential):
  def __init__(self, nin, num_dense_layers, growth_rate, drop_rate=0.0):
      super(DenseBlock, self).__init__()
                        
      for i in range(num_dense_layers):
          nin_dense_layer = nin + growth_rate * i
          self.add_module('dense_layer_%d' % i, dense_layer(nin=nin_dense_layer, growth_rate=growth_rate, drop_rate=drop_rate))

In [0]:
class PeleeNet(nn.Module):
    def __init__(self, growth_rate=32, num_dense_layers=[3,4,8,6], theta=1, drop_rate=0.0, num_classes=10):
        super(PeleeNet, self).__init__()
        
        assert len(num_dense_layers) == 4
        
        self.features = nn.Sequential()
        self.features.add_module('StemBlock', StemBlock())
        
        nin_transition_layer = 32
        
        for i in range(len(num_dense_layers)):
            self.features.add_module('DenseBlock_%d' % (i+1), DenseBlock(nin=nin_transition_layer, num_dense_layers=num_dense_layers[i], growth_rate=growth_rate, drop_rate=0.0))
            nin_transition_layer +=  num_dense_layers[i] * growth_rate
            
            if i == len(num_dense_layers) - 1:
                self.features.add_module('Transition_layer_%d' % (i+1), conv_bn_relu(nin=nin_transition_layer, nout=int(nin_transition_layer*theta), kernel_size=1, stride=1, padding=0, bias=False))
            else:
                self.features.add_module('Transition_layer_%d' % (i+1), Transition_layer(nin=nin_transition_layer, theta=1))
        
        #################### Miok ####################
        #self.linear = nn.Linear(nin_transition_layer, num_classes)
        self.fc_v2 = nn.Conv2d(704, num_classes, 1, 1, groups=2)
    def forward(self, x):
        stage_output = self.features(x)
        
        global_avg_pool_output = F.adaptive_avg_pool2d(stage_output, (1, 1))  
        output = self.fc_v2(global_avg_pool_output)
        output = output.view(output.size(0), -1)
        ##############################################
        return output

In [0]:
net = PeleeNet()

In [0]:
def get_training_dataloader(mean, std, batch_size=16, num_workers=2, shuffle=True):
    """ return training dataloader
    Args:
        mean: mean of cifar100 training dataset
        std: std of cifar100 training dataset
        path: path to cifar100 training python dataset
        batch_size: dataloader batchsize
        num_workers: dataloader num_works
        shuffle: whether to shuffle 
    Returns: train_data_loader:torch dataloader object
    """

    transform_train = transforms.Compose([
        #transforms.ToPILImage(),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    #cifar100_training = CIFAR100Train(path, transform=transform_train)
    cifar100_training = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
    cifar100_training_loader = DataLoader(
        cifar100_training, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)

    return cifar100_training_loader

def get_test_dataloader(mean, std, batch_size=16, num_workers=2, shuffle=True):
    """ return training dataloader
    Args:
        mean: mean of cifar100 test dataset
        std: std of cifar100 test dataset
        path: path to cifar100 test python dataset
        batch_size: dataloader batchsize
        num_workers: dataloader num_works
        shuffle: whether to shuffle 
    Returns: cifar100_test_loader:torch dataloader object
    """

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    #cifar100_test = CIFAR100Test(path, transform=transform_test)
    cifar100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    cifar100_test_loader = DataLoader(
        cifar100_test, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)
    
    return cifar100_test_loader

class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

In [24]:
transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

transform_validation = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])


transform_test = transforms.Compose([
        transforms.Resize(224),     
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))])

cifar100_training_loader = get_training_dataloader(
        CIFAR100_TRAIN_MEAN,
        CIFAR100_TRAIN_STD,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=True
    )

cifar100_test_loader = get_test_dataloader(
        CIFAR100_TRAIN_MEAN,
        CIFAR100_TRAIN_STD,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=False
    )

Files already downloaded and verified
Files already downloaded and verified


In [25]:
net.to(device)

PeleeNet(
  (features): Sequential(
    (StemBlock): StemBlock(
      (conv_3x3_first): conv_bn_relu(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (conv_1x1_left): conv_bn_relu(
        (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), groups=4, bias=False)
        (batch_norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (shuffle1): ShuffleBlock()
      (conv_3x3_left): conv_bn_relu(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (max_pool_right): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1,

In [26]:
torchsummary.summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
      conv_bn_relu-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 16, 112, 112]             128
       BatchNorm2d-6         [-1, 16, 112, 112]              32
              ReLU-7         [-1, 16, 112, 112]               0
      conv_bn_relu-8         [-1, 16, 112, 112]               0
      ShuffleBlock-9         [-1, 16, 112, 112]               0
           Conv2d-10           [-1, 32, 56, 56]           4,608
      BatchNorm2d-11           [-1, 32, 56, 56]              64
             ReLU-12           [-1, 32, 56, 56]               0
     conv_bn_relu-13           [-1, 32, 56, 56]               0
        MaxPool2d-14           [-1, 32,

In [0]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.2) #learning rate decay
#20, 40, 80
# 20 <= epoch < 40, lr = lr*0.2 
iter_per_epoch = len(cifar100_training_loader)
warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * warm)
checkpoint_path = os.path.join(CHECKPOINT_PATH, "PeleeNet", TIME_NOW)

In [0]:
def train(epoch):

    net.train()
    num_trained_data = 0
    for batch_index, (images, labels) in enumerate(cifar100_training_loader):
        if epoch <= warm:
            warmup_scheduler.step()

        images = Variable(images)
        labels = Variable(labels)

        labels = labels.cuda()
        images = images.cuda()
        
        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        n_iter = (epoch - 1) * len(cifar100_training_loader) + batch_index + 1
        num_trained_data += len(images)

In [0]:
def eval_training(epoch):
    net.eval()

    test_loss = 0.0 # cost function error
    correct = 0.0

    for (images, labels) in cifar100_test_loader:
        images = Variable(images)
        labels = Variable(labels)

        images = images.cuda()
        labels = labels.cuda()

        outputs = net(images)
        loss = loss_function(outputs, labels)
        test_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum()

    print('[', epoch, 'epoch]')
    print('Test set: Average loss: {:.4f}, Accuracy: {:.4f}'.format(
        test_loss / len(cifar100_test_loader.dataset),
        correct.float() / len(cifar100_test_loader.dataset)
    ))
    print()

    return correct.float() / len(cifar100_test_loader.dataset)

In [37]:
!ls

checkpoint  data  sample_data


In [0]:
if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{net}-{epoch}-{type}.pth')

best_acc = 0.0
for epoch in range(1, EPOCH):
    if epoch > warm:
        train_scheduler.step(epoch)

    train(epoch)
    acc = eval_training(epoch)

    #start to save best performance model after learning rate decay to 0.01 
    if epoch > MILESTONES[1] and best_acc < acc:
        torch.save(net.state_dict(), checkpoint_path.format(net="PeleeNet", epoch=epoch, type='best'))
        best_acc = acc
        continue

    if not epoch % SAVE_EPOCH:
        torch.save(net.state_dict(), checkpoint_path.format(net="PeleeNet", epoch=epoch, type='regular'))
print('Training Finish')

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[ 1 epoch]
Test set: Average loss: -0.0984, Accuracy: 0.0124



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[ 2 epoch]
Test set: Average loss: 0.0206, Accuracy: 0.0152

[ 3 epoch]
Test set: Average loss: 0.0116, Accuracy: 0.0128

[ 4 epoch]
Test set: Average loss: -0.0149, Accuracy: 0.0169

[ 5 epoch]
Test set: Average loss: 0.0335, Accuracy: 0.0187

[ 6 epoch]
Test set: Average loss: nan, Accuracy: 0.0100

[ 7 epoch]
Test set: Average loss: 0.0194, Accuracy: 0.0150

[ 8 epoch]
Test set: Average loss: -0.1195, Accuracy: 0.0100

[ 9 epoch]
Test set: Average loss: 0.0335, Accuracy: 0.0124

[ 10 epoch]
Test set: Average loss: nan, Accuracy: 0.0100

[ 11 epoch]
Test set: Average loss: -0.1162, Accuracy: 0.0100

[ 12 epoch]
Test set: Average loss: 0.0335, Accuracy: 0.0100

[ 13 epoch]
Test set: Average loss: -0.1194, Accuracy: 0.0100

[ 14 epoch]
Test set: Average loss: -0.1241, Accuracy: 0.0100

[ 15 epoch]
Test set: Average loss: 0.0138, Accuracy: 0.0100

[ 16 epoch]
Test set: Average loss: -0.0124, Accuracy: 0.0100

[ 17 epoch]
Test set: Average loss: 0.0336, Accuracy: 0.0100

[ 18 epoch]
Test